### Stanford CS231n Deep Learning for Computer Vision
#### Module 1. Neural Networks: Linear Classification

In [ ]:
# Loss function implementation
# unvectorized ver. 

def L_i(x,y,W):
    """
    unvectorized version. Compute the multiclass svm loss for a single example (x,y)
    - x: a column vector representing an image (e.g. 3073*1 in CIFAR-10) with an appended bias dimension in the 3073-rd position (i.e. bias trick)
    - y: an integer giving index of correct class
    - W: the weight matrix (e.g. 10*3073 in CIFAR-10)
    """
    delta: 1.0 
    scores=W.dot(x) # scores become of size 10*1, the scores for each class
    correct_class_score=scores[y]
    D=W.shape[0] # number of classes, e.g. 10
    loss_i=0.0
    
    for j in range(D): #iterate over all wrong classes
        if j==y:
            continue # skipping for the true class to only loop over incorrect classes
        loss_i+=max(0, scores[j]-correct_class_score+delta)
        return loss_i
    
    
    

In [ ]:
# Half-vectorized ver. 
def L_i_vectorized(x,y,W):
    """ 
    A faster half-vectorized implementation. half-vectorized refers to the fact that for a single example the implementation contains no for loops, but there is still one loop over the examples (outside this function)
    """ 
    delta=1.0
    scores=W.dot(x)
    # compute the margins for all calsses in one vector operation
    margins=np.maximum(0, scores-scores[y]+delta) 
    # on y-th position scores[y]-scores[y] canceld and gave delta. We want to ignore the y-th position and only consider margin on max wrong class
    margins[y]=0
    loss_i=np.sum(margins)
    return loss_i

### Optimization
##### Strategy 1: A first very bad idea solution: Random search 

In [ ]:
# X_train: the data where each column is a example (e.g. 3073(pixels as rows) * 50,000(images as cols))
# Y_train: the labels (e.g. 1D array of 50,000(ground truth labels))
# L: evaluates the loss function
# W: weight matrix (10(# of class)*3073(features)) 
# each row of W corresponds to the template of each class(Wk)
import numpy as np

bestloss=float("inf")
for num in range(1000):
    W=np.random.randn(10, 3073)*0.0001 # generate random parameters for each iteration
    loss= L_i(X_train, Y_train, W) # get the loss over the entire training set 
    if loss<bestloss: 
        bestloss=loss
        bestW=W
        
        print("in attempt %d the loss was %f, best %f" %(num, loss, bestloss))
        
# Assume X_test is [3073 x 10000], Y_test [10000 x 1]
scores = bestW.dot(Xte_cols) # 10 x 10000, the class scores for all test examples
# find the index with max score in each col (the predicted class)
Yte_predict = np.argmax(scores, axis=0)
# and calculate accuracy (fraction of predictions that are correct)
np.mean(Yte_predict==Yte)

##### Strategy 2: Random Local Search 
##### Start with random weights and iteratively refine them over time to get lower loss

In [ ]:
W=np.random.randn(10,3073)*0.001 # generate random starting W 
bestloss=float("inf")
for i in range(1000):
    step_size=0.0001
    Wtry=W+np.random.randn(10,3073)*step_size
    loss=L_i(X_train, Y_train, Wtry)
    if loss<bestloss:
        W=Wtry
        bestloss=loss
    print("iter %d loss is %f" % (i, bestloss))
        

#### Strategy 3: Following the Gradient 

There are two ways to compute the gradient:
1) Numerical gradient: A slow, approximate but easy way
2) Analytic gradient: A fast, exact but more error-prone way that requires calculus

In [ ]:
# Computing the gradient numerically with finite differences

def eval_numerical_gradient(f,x):
    """
    a naive implementation of numerical gradient of f @ x
    - f should be a function that takes a single argument
    - x is the point (numpy array) to evaluate the gradient at 
    """
    
    fx=f(x) # evaluate function value at original point
    grad=np.zeros(x.shape)
    h=0.00001
    
    #iterate over all indexes in x
    it=np.nditer(x, flags=["multi_index"], op_flags=["readwrite"])
    while not it.finished: 
        # evaluate function at x+h
        ix=it.multi_index
        old_value=x[ix]
        x[ix]=old_value+h
        fxh=f(x) # evaluate f(x+h)
        x[ix]=old_value #restore to previous value *****
        
        #compute the partial derivative 
        grad[ix]=(fxh-fx)/h
        it=iternext()
        
    return grad
    


- The gradient tells us **which direction** decreases loss fastest, but not **how far to move(step size)**.
- A small step size yields steady but slow progress; a large step can overshoot and worsen loss.
- Finite‐difference gradients take O(D) loss evaluations (one per parameter) and don’t scale to millions of weights.
- In practice, we rely on efficient automatic differentiation (autograd) instead of numeric gradients.

- Analytic gradient computation is exact and easy to implement once you’ve derived the formulas.

#### Gradient Descent

In [ ]:
# Vanilla Gradient Descent 

while True:
    weights_grad=evaluate_gradient(loss_ftn, data, weights)
    weights+= -step_size*weights_grad # perform parameter update --> towards negative gradient direction

In [ ]:
# Vanilla Minibatch Gradient Descent
# In practice, the dataset would not contain duplicate images, the gradient from a mini-batch is a good approximation of the gradient of the full objective.
while True: 
    data_batch=sample_training_data(data, 256) #sample 256 examples
    weights_grad=evaluate_gradient(loss_ftn, data_batch, weights)
    weights+= -step_size*weights_grad 

## Section Summary

In this section, we covered the end-to-end process of optimizing a high-dimensional loss landscape:

1. **Loss as a “Bowl-Shaped” Landscape**  
   - We view the loss function as a high-dimensional terrain and our goal as finding its bottom.  
   - The SVM loss is piece-wise linear but globally bowl-shaped.

2. **Iterative Refinement**  
   - Start from a random weight initialization.  
   - Gradually refine the weights step by step to reduce the loss.

3. **Role of the Gradient**  
   - The gradient points in the direction of steepest increase.  
   - To minimize loss, we move in the **negative gradient** direction.

4. **Numerical vs. Analytic Gradients**  
   - **Numerical gradients** use finite differences \((f(x+h) - f(x)) / h\): simple but \(O(D)\) expensive.  
   - **Analytic gradients** are exact and efficient but require correct mathematical derivation.  
   - In practice we implement the analytic form and perform a **gradient check** against the numerical result.

5. **Importance of Step Size (Learning Rate)**  
   - Too small ⇒ steady but slow convergence.  
   - Too large ⇒ risk of overshooting or divergence.  
   - Choosing and scheduling the right learning rate is critical.

6. **Vanilla Gradient Descent Loop**  
   ```python
   while not converged:
       grad = compute_gradient(loss_fn, data, weights)
       weights -= learning_rate * grad